In [3]:
import pyjacob
import pandas as pd
import cantera as ct
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt

In [4]:
# Setting of plot

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
# Setting of font
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rcParams["font.size"] = 12 

plt.rcParams["mathtext.fontset"] = 'stix'

## Please use these code while plotting
# plt.xlabel('$1/a_{\mathrm{1}}$ [s]', fontsize=14)
# plt.ylabel('$T_{\mathrm{max}}$ [K]', fontsize=14)
# plt.tick_params(which='both', direction='in')
# plt.legend(frameon=False)
# ax = plt.gca()
# ax.spines["right"].set_color("none")
# ax.spines["top"].set_color("none")

## Please use this code while saving
# name_fig = 'IDT_CH4_f0.5_20atm_GRI_O6e02_CONP'
# plt.savefig(name_fig + '.eps', format = 'eps', bbox_inches="tight")
# plt.savefig(name_fig + '.png', format = 'png', bbox_inches="tight", dpi=100)

In [5]:
#create gas from original mechanism file gri30.cti
gas = ct.Solution('NH3_Tamaoki.yaml')
#reorder the gas to match pyJac
n2_ind = gas.species_index('N2')
specs = gas.species()[:]
gas = ct.Solution(thermo='IdealGas', kinetics='GasKinetics',
        species=specs[:n2_ind] + specs[n2_ind + 1:] + [specs[n2_ind]],
        reactions=gas.reactions())

In [6]:
def get_eigvals(gas, Tin, Taf):

    #setup the state vector
    y = np.zeros(gas.n_species)
    y[0] = gas.T
    y[1:] = gas.Y[:-1]

    #create a dydt vector
    dydt = np.zeros_like(y)
    pyjacob.py_dydt(0, P, y, dydt)

    #create a jacobian vector
    jac = np.zeros(gas.n_species * gas.n_species)

    #evaluate the Jacobian
    pyjacob.py_eval_jacobian(0, P, y, jac)

    # reshape, and transpose
    jac = jac.reshape((gas.n_species,gas.n_species)).T

    #rescaling using (Taf - Tin)
    jac[0,:] *= 1.0/(Taf -Tin)
    jac[:,0] *= (Taf -Tin)

    # get eigan values
    eigvals = LA.eigvals(jac)

    return eigvals

#set the gas state
Tin = 1000
P = ct.one_atm
# C2H4 + 3O2 = 2CO2 + 2H2O
gas.TPY = Tin, P, "NH3:1.0, O2:3, N2:11.285"

# get the diabatic flame temperature Taf
oldstate = gas.TPY
gas.equilibrate('HP')
Taf = gas.T
gas.TPY=oldstate

evals = get_eigvals(gas, Tin, Taf)
ser = np.sort(evals.real)
ser

array([-1.08116392e+09, -1.76427490e+08, -3.67957861e+06, -2.55534051e+06,
       -5.58537047e+05, -3.10070493e+05, -2.41445372e+05, -1.54809704e+05,
       -1.17094391e+05, -8.05809210e+04, -1.31191525e+04, -1.11564491e+04,
       -8.34486935e+02, -6.58327613e+02, -3.68024696e+02, -1.54382972e+02,
       -8.74886113e+01, -2.16184002e+01, -2.08412892e+01, -6.70521241e+00,
       -1.50847696e+00, -1.00318163e-01, -2.96902986e-03, -1.48959412e-03,
       -4.35153962e-05, -2.29088776e-08, -2.84926268e-11, -7.98032036e-13,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.81268880e-07,
        1.70664836e+01])

In [7]:
gas.Y

array([0.        , 0.        , 0.        , 0.19627085, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.06542362, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.73830553])